In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys

In [ ]:
project_path = '/content/drive/My Drive/ML project 2/'
sys.path.append(project_path)

In [ ]:
import os
import torchvision.transforms as T
import torch
import json
from levin.train_evaluate import TrainAndEvaluate
from levin.models.segformer.segformer_dataset import SegformerRoadSegmentationDataset
from levin.models.segformer.segformer_b3 import SegFormer
from levin.loss_functions import ComboLoss, DiceLoss, TverskyLoss, WeightedBCEWithLogitsLoss
from transformers import SegformerImageProcessor
from torchvision.transforms.functional import InterpolationMode
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [ ]:
image_dir = os.path.join(project_path, "data", "training", "augmented", "images")
mask_dir = os.path.join(project_path, "data", "training", "augmented", "masks")

external_image_dir = os.path.join(project_path, "data", "training", "complete_data_augmented", "images")
external_mask_dir = os.path.join(project_path, "data", "training", "complete_data_augmented", "masks")

chicago_image_dir = os.path.join(project_path, "data", "training", "chicago_data_augmented", "images")
chicago_mask_dir = os.path.join(project_path, "data", "training", "chicago_data_augmented", "masks")

In [ ]:
# Load files for original augmented, external images and chicago images
image_filenames = sorted(os.listdir(image_dir))
mask_filenames = sorted(os.listdir(mask_dir))

external_image_filenames = sorted(os.listdir(external_image_dir))
external_mask_filenames = sorted(os.listdir(external_mask_dir))

chicago_image_filenames = sorted(os.listdir(chicago_image_dir))
chicago_mask_filenames = sorted(os.listdir(chicago_mask_dir))

In [ ]:
import re
# Clean images and masks for external and chicago data
directories = [external_image_dir, external_mask_dir, chicago_image_dir, chicago_mask_dir]

pattern = re.compile(r".*\(\d+\)\.png$")

# Loop through each directory and remove matching files
for directory in directories:
    for filename in os.listdir(directory):
        if pattern.match(filename):  # Check if the filename matches the pattern
            file_path = os.path.join(directory, filename)
            os.remove(file_path)  # Delete the file
            print(f"Deleted: {file_path}")

In [ ]:
print(f"Number of exernal images: {len(external_image_filenames)}, number of external masks: {len(external_mask_filenames)}")
print(f"Number of chicago images: {len(chicago_image_filenames)}, number of chicago masks: {len(chicago_mask_filenames)}")


Number of exernal images: 8660, number of external masks: 8660
Number of chicago images: 2785, number of chicago masks: 2785


In [ ]:
image_dir_filenames = set(os.listdir(image_dir))
mask_dir_filenames = set(os.listdir(mask_dir))

# Check for images without masks
images_without_masks = image_dir_filenames - mask_dir_filenames
# Check for masks without images
masks_without_images = mask_dir_filenames - image_dir_filenames

# Print results
print("Images without corresponding masks:")
print(images_without_masks)

print("Masks without corresponding images:")
print(masks_without_images)

Images without corresponding masks:
set()
Masks without corresponding images:
set()


In [ ]:
### Original augmented data
# Create train test and validation set for original augmented data
train_images, test_images, train_masks, test_masks = train_test_split(
    image_filenames, mask_filenames, test_size=0.2, random_state=42
)
train_images, validation_images, train_masks, validation_masks = train_test_split(
    train_images, train_masks, test_size=0.1, random_state=42)

print(f"Number of training images: {len(train_images)}")
print(f"Number of validation images: {len(validation_images)}")
print(f"Number of test images: {len(test_images)}")

# Define datasets for train and test sets
train_dataset = SegformerRoadSegmentationDataset(image_dir, mask_dir, train_images, train_masks)
test_dataset = SegformerRoadSegmentationDataset(image_dir, mask_dir, test_images, test_masks)
validation_dataset = SegformerRoadSegmentationDataset(image_dir, mask_dir, validation_images, validation_masks)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)
validation_loader = DataLoader(validation_dataset, batch_size=8, shuffle=False, num_workers=2)


### Complete augmented original and external data
# Create train test and validation set for external data
external_train_images, external_test_images, external_train_masks, external_test_masks = train_test_split(
    external_image_filenames, external_mask_filenames, test_size=0.2, random_state=42
)
external_train_images, external_validation_images, external_train_masks, external_validation_masks = train_test_split(
    external_train_images, external_train_masks, test_size=0.1, random_state=42)

print(f"Number of external training images: {len(external_train_images)}")
print(f"Number of external validation images: {len(external_validation_images)}")
print(f"Number of external test images: {len(external_test_images)}")

# Define datasets for train and test sets
external_train_dataset = SegformerRoadSegmentationDataset(external_image_dir, external_mask_dir, external_train_images, external_train_masks)
external_test_dataset = SegformerRoadSegmentationDataset(external_image_dir, external_mask_dir, external_test_images, external_test_masks)
external_validation_dataset = SegformerRoadSegmentationDataset(external_image_dir, external_mask_dir, external_validation_images, external_validation_masks)

external_train_loader = DataLoader(external_train_dataset, batch_size=8, shuffle=True, num_workers=2)
external_test_loader = DataLoader(external_test_dataset, batch_size=8, shuffle=False, num_workers=2)
external_validation_loader = DataLoader(external_validation_dataset, batch_size=8, shuffle=False, num_workers=2)


### Augmented original and chicago data
# Create train test and validation set for external chicago data
chicago_train_images, chicago_test_images, chicago_train_masks, chicago_test_masks = train_test_split(
    chicago_image_filenames, chicago_mask_filenames, test_size=0.2, random_state=42
)
chicago_train_images, chicago_validation_images, chicago_train_masks, chicago_validation_masks = train_test_split(
    chicago_train_images, chicago_train_masks, test_size=0.1, random_state=42)

print(f"Number of chicago training images: {len(chicago_train_images)}")
print(f"Number of chicago validation images: {len(chicago_validation_images)}")
print(f"Number of chicago test images: {len(chicago_test_images)}")

# Define datasets for train and test sets
chicago_train_dataset = SegformerRoadSegmentationDataset(chicago_image_dir, chicago_mask_dir, chicago_train_images, chicago_train_masks)
chicago_test_dataset = SegformerRoadSegmentationDataset(chicago_image_dir, chicago_mask_dir, chicago_test_images, chicago_test_masks)
chicago_validation_dataset = SegformerRoadSegmentationDataset(chicago_image_dir, chicago_mask_dir, chicago_validation_images, external_validation_masks)

chicago_train_loader = DataLoader(chicago_train_dataset, batch_size=8, shuffle=True, num_workers=2)
chicago_test_loader = DataLoader(chicago_test_dataset, batch_size=8, shuffle=False, num_workers=2)
chicago_validation_loader = DataLoader(chicago_validation_dataset, batch_size=8, shuffle=False, num_workers=2)

Number of training images: 360
Number of validation images: 40
Number of test images: 100
Number of external training images: 6235
Number of external validation images: 693
Number of external test images: 1732
Number of chicago training images: 2005
Number of chicago validation images: 223
Number of chicago test images: 557


In [ ]:
# Test different loss functions for original augmented data
root_path = os.path.join(project_path, "levin")
save_path = os.path.join(root_path, "trained_models", "segformer_b3")
print(f"Created save path {save_path}")
os.makedirs(save_path, exist_ok=True)

loss_functions = [torch.nn.BCEWithLogitsLoss(), WeightedBCEWithLogitsLoss(), DiceLoss(), ComboLoss(), TverskyLoss()]
best_f1 = 0
best_threshold = 0
best_epoch = 0
best_loss_function = None
best_data = "original"
for criterion in loss_functions:
  print(f"Training model with {criterion.__class__.__name__}")
  # Train and evaluate
  model = SegFormer()
  train_eval = TrainAndEvaluate(model, train_loader, validation_loader, test_loader, criterion, 10, save_path, save=False)
  f1, threshold, epoch = train_eval.run()
  if f1 > best_f1:
    best_f1 = f1
    best_threshold = threshold
    best_epoch = epoch
    best_loss_function = criterion

print(f"Best F1 score: {best_f1}")
print(f"Best threshold: {best_threshold}")
print(f"Best epoch: {best_epoch}")
print(f"Best loss function: {best_loss_function.__class__.__name__}")

Created save path /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3
Training model with BCEWithLogitsLoss


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b3-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 768, 1, 1]) in the checkpoint and torch.Size([1, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
You sho

Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/BCEWithLogitsLoss/trained_model_epoch1.pt
Epoch 1/10, Training Loss: 0.4575, Validation Loss: 0.2637
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/BCEWithLogitsLoss/trained_model_epoch2.pt
Epoch 2/10, Training Loss: 0.1646, Validation Loss: 0.1573
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/BCEWithLogitsLoss/trained_model_epoch3.pt
Epoch 3/10, Training Loss: 0.1158, Validation Loss: 0.1337
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/BCEWithLogitsLoss/trained_model_epoch4.pt
Epoch 4/10, Training Loss: 0.0908, Validation Loss: 0.1307
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/BCEWithLogitsLoss/trained_model_epoch5.pt
Epoch 5/10, Training Loss: 0.0743, Validation Loss: 0.1246
Model saved to /content/drive/My Drive/ML project 2/levin/trained

/content/drive/My Drive/ML project 2/levin/train_evaluate.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.model.load_state_dict(torch.load(best_model_path))

Best model loaded from /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/BCEWithLogitsLoss/trained_model_epoch5.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/BCEWithLogitsLoss/trained_model_epoch1.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/BCEWithLogitsLoss/trained_model_epoch2.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/BCEWithLogitsLoss/trained_model_epoch3.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/BCEWithLogitsLoss/trained_model_epoch4.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/BCEWithLogitsLoss/trained_model_epoch6.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/BCEWithLogitsLoss/trained_model_epoch7.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/BCEWithLog

...

Best model loaded from /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/ComboLoss/trained_model_epoch6.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/ComboLoss/trained_model_epoch1.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/ComboLoss/trained_model_epoch2.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/ComboLoss/trained_model_epoch3.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/ComboLoss/trained_model_epoch4.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/ComboLoss/trained_model_epoch5.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/ComboLoss/trained_model_epoch7.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/ComboLoss/trained_model_epoch8.pt
Deleted model: /content/drive/My

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b3-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 768, 1, 1]) in the checkpoint and torch.Size([1, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLoss/trained_model_epoch1.pt
Epoch 1/10, Training Loss: 0.5734, Validation Loss: 0.3551
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLoss/trained_model_epoch2.pt
Epoch 2/10, Training Loss: 0.2278, Validation Loss: 0.1673
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLoss/trained_model_epoch3.pt
Epoch 3/10, Training Loss: 0.1654, Validation Loss: 0.1465
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLoss/trained_model_epoch4.pt
Epoch 4/10, Training Loss: 0.1395, Validation Loss: 0.1392
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLoss/trained_model_epoch5.pt
Epoch 5/10, Training Loss: 0.1337, Validation Loss: 0.1513
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLo

/content/drive/My Drive/ML project 2/levin/train_evaluate.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.model.load_state_dict(torch.load(best_model_path))

Best model loaded from /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLoss/trained_model_epoch10.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLoss/trained_model_epoch1.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLoss/trained_model_epoch2.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLoss/trained_model_epoch3.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLoss/trained_model_epoch4.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLoss/trained_model_epoch5.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLoss/trained_model_epoch6.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/TverskyLoss/trained_model_epoch7.pt
Deleted model: 

In [ ]:
root_path = os.path.join(project_path, "levin")
best_loss_function = torch.nn.BCEWithLogitsLoss()
best_f1 = 0.8933

In [ ]:
# Train and evaluate the model with the best loss function on external data
external_save_path = os.path.join(root_path, "trained_models", "segformer_b3", "external")
print("Created external save path")
os.makedirs(external_save_path, exist_ok=True)
model = SegFormer()
train_eval = TrainAndEvaluate(model, external_train_loader, external_validation_loader, external_test_loader, best_loss_function, 10, external_save_path)
f1, threshold, epoch = train_eval.run()
if f1 > best_f1:
    best_f1 = f1
    best_threshold = threshold
    best_epoch = epoch
    best_data = "external"

Created external save path


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b3-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 768, 1, 1]) in the checkpoint and torch.Size([1, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/external/BCEWithLogitsLoss/trained_model_epoch1.pt
Epoch 1/10, Training Loss: 0.2210, Validation Loss: 0.1694
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/external/BCEWithLogitsLoss/trained_model_epoch2.pt
Epoch 2/10, Training Loss: 0.1437, Validation Loss: 0.1460
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/external/BCEWithLogitsLoss/trained_model_epoch3.pt
Epoch 3/10, Training Loss: 0.1244, Validation Loss: 0.1383
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/external/BCEWithLogitsLoss/trained_model_epoch4.pt
Epoch 4/10, Training Loss: 0.1105, Validation Loss: 0.1357
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/external/BCEWithLogitsLoss/trained_model_epoch5.pt
Epoch 5/10, Training Loss: 0.0975, Validation Loss: 0.1307
Model saved to /cont

/content/drive/My Drive/ML project 2/levin/train_evaluate.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.model.load_state_dict(torch.load(best_model_path))

Best model loaded from /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/external/BCEWithLogitsLoss/trained_model_epoch5.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/external/BCEWithLogitsLoss/trained_model_epoch1.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/external/BCEWithLogitsLoss/trained_model_epoch2.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/external/BCEWithLogitsLoss/trained_model_epoch3.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/external/BCEWithLogitsLoss/trained_model_epoch4.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/external/BCEWithLogitsLoss/trained_model_epoch6.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/external/BCEWithLogitsLoss/trained_model_epoch7.pt
Deleted model: /content/drive/My 

In [ ]:
# Train and evaluate the model with the best loss function on chicago data
chicago_save_path = os.path.join(root_path, "trained_models", "segformer_b3", "chicago")
print("Created chicago save path")
os.makedirs(chicago_save_path, exist_ok=True)
model = SegFormer()
train_eval = TrainAndEvaluate(model, external_train_loader, external_validation_loader, external_test_loader, best_loss_function, 10, chicago_save_path)
f1, threshold, epoch = train_eval.run()
if f1 > best_f1:
    best_f1 = f1
    best_threshold = threshold
    best_epoch = epoch
    best_data = "chicago"

Created chicago save path


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/6.89k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/190M [00:00<?, ?B/s]

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b3-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 768, 1, 1]) in the checkpoint and torch.Size([1, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/chicago/BCEWithLogitsLoss/trained_model_epoch1.pt
Epoch 1/10, Training Loss: 0.2157, Validation Loss: 0.1634
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/chicago/BCEWithLogitsLoss/trained_model_epoch2.pt
Epoch 2/10, Training Loss: 0.1437, Validation Loss: 0.1509
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/chicago/BCEWithLogitsLoss/trained_model_epoch3.pt
Epoch 3/10, Training Loss: 0.1244, Validation Loss: 0.1385
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/chicago/BCEWithLogitsLoss/trained_model_epoch4.pt
Epoch 4/10, Training Loss: 0.1096, Validation Loss: 0.1367
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/chicago/BCEWithLogitsLoss/trained_model_epoch5.pt
Epoch 5/10, Training Loss: 0.0977, Validation Loss: 0.1341
Model saved to /content/d

/content/drive/My Drive/ML project 2/levin/train_evaluate.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.model.load_state_dict(torch.load(best_model_path))

Best model loaded from /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/chicago/BCEWithLogitsLoss/trained_model_epoch5.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/chicago/BCEWithLogitsLoss/trained_model_epoch1.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/chicago/BCEWithLogitsLoss/trained_model_epoch2.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/chicago/BCEWithLogitsLoss/trained_model_epoch3.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/chicago/BCEWithLogitsLoss/trained_model_epoch4.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/chicago/BCEWithLogitsLoss/trained_model_epoch6.pt
Deleted model: /content/drive/My Drive/ML project 2/levin/trained_models/segformer_b3/chicago/BCEWithLogitsLoss/trained_model_epoch7.pt
Deleted model: /content/drive/My Drive/M

In [ ]:
results = {"best_criterion": best_loss_function,
           "best_f1": best_f1,
           "best_threshold": best_threshold,
           "best_epoch": best_epoch,
           "best_data": best_data
           }

results_path = os.path.join(root_path, "models", "segformer", "train_results.json")
with open(results_path, "w") as json_file:
   json.dump(results, json_file, indent=4)

print(f"Best data: {best_data}, best_criterion: {best_loss_function} with f1: {best_f1}, best epoch: {best_epoch}, best_threshold: {best_threshold}")
print("Finished training, everything saved")